# Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import preprocessing ,model_selection,neighbors,svm
from sklearn import metrics
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import xgboost, numpy, textblob, string

# Data Preprocessing And Cleaning

Loading Data

In [2]:
df=pd.read_csv('Datasets/spam.csv')
df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1,inplace=True)
category={"ham": 0, "spam":1}
df["v1"].replace(category, inplace=True)
df.rename({"v1": "is_spam", "v2": "message"},axis=1, inplace=True)
df.head()

,is_spam,message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


Cleaning Process

In [7]:
s=df.iloc[2,1]

In [8]:
s

"Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"

In [4]:
def clean(s):
    s=re.sub(r'[^\w\s]', '',s)
    stop_words=set(stopwords.words('english'))
    tokens= word_tokenize(s)
    cleaned = [word for word in tokens if word not in stop_words]
    return " ".join(cleaned)
df['message']=df['message'].apply(clean)

Splitting the dataset

In [5]:
X=df.message
y=df.is_spam
train_x, test_x, train_y, test_y = model_selection.train_test_split(X,y)

In [6]:
train_x

4768    watever reLation u built dis world thing remai...
5541                                        Yeah jus rite
3583    Hi 07734396839 IBH Customer Loyalty Offer The ...
1221                                         Prakesh know
1955      Aight Ill grab something eat text youre back mu
                              ...                        
4019                       University southern california
2338                                      Alright see bit
482                                       Watching tv lor
4975                  You gorgeous keep pix cumming thank
5285    URGENT You 1 week FREE membership 100000 Prize...
Name: message, Length: 4179, dtype: object

Feature engineering

In [16]:
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', ngram_range=(2,3))
tesing=tfidf_vect_ngram_chars.fit(X)
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xtest_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(test_x)

Saving the tfidf fit feature

In [17]:
import pickle

In [19]:
pickle.dump(tesing, open("feature.pkl", "wb"))

Feature engineering usinb vectorizer

# Defining The Model

XGBOOST

In [56]:
xg_clf=xgboost.XGBClassifier(use_label_encoder=False)

In [57]:
model=xg_clf.fit(xtrain_tfidf_ngram_chars,train_y)
y_predict=xg_clf.predict(xtest_tfidf_ngram_chars)
score=metrics.accuracy_score(test_y,y_predict)
print("Xgboost on tfidf char level",score)

[14:36:06] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Xgboost on tfidf char level 0.9856424982053122


# testing for single input

In [23]:
def clean_input(s):
    s=re.sub(r'[^\w\s]', '',s)
    stop_words=set(stopwords.words('english'))
    tokens= word_tokenize(s)
    cleaned = [word for word in tokens if word not in stop_words]
    return " ".join(cleaned)

In [110]:
s="WINNER As valued network customer selected receivea 900 prize reward To claim call 09061701461 Claim code KL341 Valid 12 hours"

In [111]:
test=[clean_input(s)]
tfidf_test= TfidfVectorizer(analyzer='char', ngram_range=(2,3))
tfidf_test.fit(X)
tf_test = tfidf_test.transform(test) 

In [112]:
tf_test

<1x10436 sparse matrix of type '<class 'numpy.float64'>'
	with 210 stored elements in Compressed Sparse Row format>

In [116]:
xg_clf.predict(tf_test)[0]

1

# Pickling the Model

In [ ]:
import pickle
pickle.dump(regressor, open('model.pkl','wb'))